In [6]:
import numpy as np
import pandas as pd
import yfinance as yf

class Indicators:
    def __init__(self, data):
        self.data = data  # DataFrame with OHLC (Open, High, Low, Close) prices
        self.close = data['Close']
    
    
class RSI(Indicators):
    def __init__(self, data, period=14):
        super().__init__(data)
        self.period = period

    def RSI(self):
        
        '''
    
        close: (m,) closing prices
        length: (int) duration to be considered for RSI calculation

        Returns:
        RSI: (m)  list with RSI values
    
    '''
        close=self.close
        length=self.period
        differ = close.diff()
        gain, loss = differ.copy(), differ.copy()
        gain[gain < 0] = 0
        loss[loss > 0] = 0

        avggain = gain.rolling(window=length).mean()
        avgloss = -loss.rolling(window=length).mean()
        rsi= 100.0 - (100.0*avgloss / (avggain+avgloss))

        self.data['RSI']=rsi
        self.data['RSISignal']=self.data['RSI'].apply(lambda x: 1 if x > 65 else (-1 if x < 35 else 0))
        return self
    
    

    
               
class SMA_EMA(Indicators):
    def __init__(self, data, period=50):
        super().__init__(data)
        self.period = period

    def SMA(self):
        '''
        Args:
        close: (m,) pandas series of closing prices
        length: (int) duration to be considered for SMA calculation

        Returns:
        SMA: pandas series with SMA values, including expanding SMA for the first (length-1) rows
        '''
        close=self.close
        length=self.period 
        expanding_sma = close.expanding().mean()

        rolling_sma = close.rolling(window=length).mean()

        SMA = expanding_sma.copy()  
        SMA[length-1:] = rolling_sma[length-1:]  

        self.data['SMA']=SMA
        return self
    
    def EMA(self):
        '''
        Args:
        close: (m,) pandas Series of closing prices
        length: (int) duration to be considered 

        Returns:
        EMA: pandas Series with EMA values

        '''
        close=self.close
        length=self.period 
        EMA = close.ewm(span=length,adjust=False).mean()
        self.data['EMA']=EMA
        return self
        

class MACD(Indicators):
    def __init__(self, data):
        super().__init__(data)

    def MACD(self):
        EMA12 = self.data['Close'].ewm(span=12, adjust=False).mean()
        EMA26 = self.data['Close'].ewm(span=26, adjust=False).mean()
        self.data['MACD'] = EMA12 - EMA26
        self.data['Signal_Line'] = self.data['MACD'].ewm(span=9, adjust=False).mean()

        self.data['MACDSignal'] = 0  
        for idx, row in self.data.iterrows():
            if row['MACD'] > row['Signal_Line']:
                self.data.at[idx, 'MACDSignal'] = 1
            elif row['MACD'] < row['Signal_Line']:
                self.data.at[idx, 'MACDSignal'] = -1



        return self

    
class BB(SMA_EMA):
    
    def __init__(self, data):
        super().__init__(data)
     
    @staticmethod 
    def std(close,length):
        '''
        Args:
        close: (m,) pandas series of closing prices
        length: (int) duration to be considered for std calculation

        Returns:
        std: pandas series with standard deviation
        '''
        if isinstance(close, pd.DataFrame):
            close = close.squeeze()  # Convert single-column DataFrame to Series
        
        expanding_std = close.expanding().std()
        rolling_std = close.rolling(window=length).std()

        std = expanding_std.copy()  
        std[length-1:] = rolling_std[length-1:]  

        return std
    
           
    def bollingerbands(self):
        self.SMA()
        self.data['20 Day MA']= self.data['SMA']
        self.data['Upper']=self.data['20 Day MA']+2*BB.std(self.data['Close'], 20)
        self.data['Lower']=self.data['20 Day MA']-2*BB.std(self.data['Close'], 20)
        self.data['BBSignal']=self.data.apply(lambda x: 1 if x['Close']<x['Lower']*1.10 else(-1 if x['Close']>x['Upper']*0.90 else 0 ),axis=1)
            
        return self



class TradingStrategy:
    def __init__(self, data):
        self.data = data
        self.data=self.data[self.data['Volume']!=0]
    
    
    
    def apply_strategy(self,tradebook):
        add={}
        self.data['Position Signal']=0

        #Note:
        
        #0 stands for no position
        #1 stands for long
        #-1 stands for short
        
        
        
        rsi = RSI(self.data).RSI()  
        self.data = rsi.data  

        macd = MACD(self.data).MACD()  
        self.data = macd.data  

        sma_ema = SMA_EMA(self.data, period=50).SMA().EMA()
        self.data = sma_ema.data  

        bb = BB(self.data).bollingerbands()  
        self.data = bb.data 
        print("Columns in self.data:", self.data.columns)
        
        
        for i in range(len(self.data)):
                if ((self.data.iloc[i]['RSISignal'] ==1 and self.data.iloc[i]['BBSignal'] == 1)):
                    if self.data.iloc[i]['Position Signal']==0:
                        #start a long trade
                        add['entry_date']=self.data.index[i]
                        add['entry_price']= self.data.iloc[i]['Close']
                        add['open']=self.data.iloc[i]['Open']
                        add['high']=self.data.iloc[i]['High']
                        add['low']=self.data.iloc[i]['Low']
                        add['trade_type']=1
                        self.data.iloc[i:,self.data.columns.get_loc('Position Signal')]=1
                    '''elif self.data.iloc[i]['Position Signal']==-1:
                        #exit the trade; stoploss
                        self.data.iloc[i:,self.data.columns.get_loc('Position Signal')]=0
                        add['exit_date']=self.data.index[i]
                        add['exit_price']=self.data.iloc[i]['Close']
                        tradebook.append(add)
                        add={}'''
                elif (self.data.iloc[i]['RSISignal'] == -1 and (self.data.iloc[i]['BBSignal'] == -1)):
                    if self.data.iloc[i]['Position Signal']==0:
                        #start a short trade
                        add['entry_date']=self.data.index[i]
                        add['open']=self.data.iloc[i]['Open']
                        add['high']=self.data.iloc[i]['High']
                        add['low']=self.data.iloc[i]['Low']
                        add['entry_price'] = self.data.iloc[i]['Close']
                        add['trade_type']=-1
                        self.data.iloc[i:,self.data.columns.get_loc('Position Signal')]=-1
                    '''elif self.data.iloc[i]['Position Signal']==1:
                        #exit the trade; stoploss for a long trade
                        self.data.iloc[i:,self.data.columns.get_loc('Position Signal')]=0 
                        add['exit_date']=self.data.index[i]
                        add['exit_price']=self.data.iloc[i]['Close']
                        tradebook.append(add)
                        add={} ''' 

                if (self.data.iloc[i]['RSI']<50 and self.data.iloc[i]['Position Signal']==1):
                    #exit
                    self.data.iloc[i:,self.data.columns.get_loc('Position Signal')]=0 
                    add['exit_date']=self.data.index[i]
                    add['exit_price']=self.data.iloc[i]['Close']
                    tradebook.append(add)
                    add={}  

                elif (self.data.iloc[i]['RSI']>50 and self.data.iloc[i]['Position Signal']==-1):
                    #exit
                    self.data.iloc[i:,self.data.columns.get_loc('Position Signal')]=0
                    add['exit_date']=self.data.index[i]
                    add['exit_price']=self.data.iloc[i]['Close']
                    tradebook.append(add)
                    add={}   
                    
        return self 


tradebook=[]
ticker=yf.Ticker('TSLA')
df=ticker.history(interval="1d",period="7y")
df = TradingStrategy(df).apply_strategy(tradebook).data

df.to_csv('dataframe.csv',index=False)        
tradebook_df=pd.DataFrame(tradebook)
tradebook_df.to_csv('trade_book.csv', index=False)
signals_df=pd.DataFrame(df['Position Signal'])
signals_df.to_csv('signals.csv',index=False)

Columns in self.data: Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'Position Signal', 'RSI', 'RSISignal', 'MACD', 'Signal_Line',
       'MACDSignal', 'SMA', 'EMA', '20 Day MA', 'Upper', 'Lower', 'BBSignal'],
      dtype='object')
